In [1]:
import torch
import torchvision
import torchvision.transforms as tf
from xml.etree import ElementTree as et
import cv2
from PIL import Image
import glob as gb
import numpy as np
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline

from utils.load_img import LoadDataset
from utils.model import create_model

from numba import jit, cuda

def iou(a,b): #Numpy array
    a,b = np.array(a, dtype=bool),np.array(b, dtype=bool)
    overlap = a*b # Logical AND
    union = a + b # Logical OR

    IOU =  np.array(overlap, dtype=float).sum()/np.array(union, dtype=float).sum()
    return IOU

usage: ipykernel_launcher.py [-h] [-bs BATCHSIZE] [-img IMGSIZE] [-e EPOCH]
                             [--data DATA] [--backbone BACKBONE]
                             [--weights WEIGHTS] [--model MODEL]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-57a238c3-b686-49e0-81ec-fbbdcb663352.json


Just using function


In [2]:
img_d = gb.glob("data/Mouth_Dataset/test/*.jpg")
lb_d = gb.glob("data/Mouth_Dataset/test/*.xml")
print(len(img_d),len(lb_d))

RESIZE_TO = (512,512)
CLASSES = [
    'background', 'oral'
]

146 146


In [23]:
#Load model Best

model = create_model(num_classes=2,model_name="fasterrcnn",backbone="mobilelow",weights='true')
checkpoint = torch.load("./myExperiment/output21/best_model.pth",map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(torch.device('cpu')).eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(320,), max_size=640, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block): 

In [28]:

# @jit(target_backend='cuda')
def test_iou(img_d,lb_d,model):
    iou_lst = []
    ll = len(img_d)
    all_time = []
    for i ,img in enumerate(img_d[:]):
        print("Loading:",i+1,"/",ll,end="\r")
#         start_time = time.time()
        #Read image
        im = Image.open(img).resize(RESIZE_TO)
        im =  np.array(im)/255
        im = [torch.tensor(im,dtype=(torch.float))]
#         end_time = time.time()
#         print("1",end_time - start_time)
        #Reshape to model pytorch model format
        im =  [im[0].reshape(3,im[0].shape[0],im[0].shape[1])]
        im = [im[0].to(torch.device('cpu'))]

        #Detection Object
        start_time = time.time()
        with torch.no_grad():
            outputs = model(im)
        end_time = time.time()
        all_time.append(end_time - start_time)
#         print("1.5",end_time - start_time)
#         start_time = time.time()
        #Get Boxes and Confident
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        boxes = boxes[scores >= 0.5].astype(np.int32) #Confident of Object detect
        arr = 0
        for j, box in enumerate(boxes[:1]): #Get only maximun confident of an model
            arr = np.zeros(RESIZE_TO)
            arr[box[0]:box[2],box[1]:box[3]] = 1
#         end_time = time.time()
#         print("2",end_time - start_time)
#         start_time = time.time()
        #Loading Ground Truth
        tree = et.parse(lb_d[i])
        root = tree.getroot()

        # box coordinates for xml files are extracted and corrected for image size given
        for member,size in zip(root.findall('object'),root.findall('size')):
            # map the current object name to `classes` list to get...
            # ... the label index and append to `labels` list
        #     labels.append(self.classes.index(member.find('name').text))

            # xmin = left corner x-coordinates
            xmin = int(member.find('bndbox').find('xmin').text)
            # xmax = right corner x-coordinates
            xmax = int(member.find('bndbox').find('xmax').text)
            # ymin = left corner y-coordinates
            ymin = int(member.find('bndbox').find('ymin').text)
            # ymax = right corner y-coordinates
            ymax = int(member.find('bndbox').find('ymax').text)

            #Get Width and Height from Original images
            width,height = int(size.find('width').text),int(size.find('height').text) 
#         end_time = time.time()
#         print("3",end_time - start_time)
        larr = np.zeros(RESIZE_TO)
        xmin_n , xmax_n = xmin/width,xmax/width
        ymin_n , ymax_n = ymin/height,ymax/height
        larr[int(ymin_n*RESIZE_TO[0]):int(ymax_n*RESIZE_TO[0]),int(xmin_n*RESIZE_TO[1]):int(xmax_n*RESIZE_TO[1])] = 1
        if arr == 0 :
            iou_lst.append(0)
        else:
            iou_lst.append(iou(arr,larr))
            
    return iou_lst,all_time

iou_lst,all_time = test_iou(img_d,lb_d,model)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
print("iou :",sum(iou_lst)/len(iou_lst),"Avg time predict :",sum(all_time)/len(all_time))

In [30]:
iou_lst

[0.018234672304439745,
 0.009302325581395349,
 0.016411395431338977,
 0.01522888112514557,
 0.01448782103171683,
 0.012757696218914672,
 0.01754583033788641,
 0.028496736271230474,
 0.03041144901610018,
 0.012508021448537649,
 0.0,
 0.0,
 0.008079053100228039,
 0.01233209214922049,
 0.013869608095629943,
 0.017270937837323006,
 0.014355166297525066,
 0.011244306680161943,
 0.01584855403348554,
 0.0022410916606489207,
 0.0,
 0.010592066838911491,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011136975672098225,
 0.0,
 0.0,
 0.02253970830965717,
 0.00971163367813212,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.021117425685249797,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0010984657494777785,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.017263227488935006,
 0.0,
 0.0,
 0.006634032172995781,
 0.00614166259765625,
 0.0063016658316633264,
 0.008568205040871935,
 0.007688340464547677,
 0.017028029496778906,
 0.01828863499099045,
 0.010745793109667206,
 0.0,
 0.018605025773195876,
 0.0,
 0.0,
 0.012146